In [ ]:
import tensorflow as tf
import numpy as np
from keras import layers, models
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Conv2D, Dropout

input_layer = Input((48,48,1))
x = Conv2D(48, 5, activation='relu')(input_layer) 
x = Dropout(0.1)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(32, activation='relu')(x)
x1 = Dense(3, activation='softmax')(x)
x2 = Dense(1, activation='sigmoid')(x)

final_output = [x1, x2]
model = tf.keras.Model(input_layer, final_output)
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 44, 44, 48)   1248        ['input_3[0][0]']                
                                                                                                  
 dropout_4 (Dropout)            (None, 44, 44, 48)   0           ['conv2d_2[0][0]']               
                                                                                                  
 flatten_2 (Flatten)            (None, 92928)        0           ['dropout_4[0][0]']              
                                                                                            

In [ ]:
#converion to int tflite using fake quantization
def representative_data_gen():
  for i in range(100):
      data = np.random.rand(1, 48, 48, 1)
      yield [data.astype(np.float32)]

model.input.set_shape((1,) + model.input.shape[1:])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpv94y4u9j/assets


INFO:tensorflow:Assets written to: /tmp/tmpv94y4u9j/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  33826      0 --:--:-- --:--:-- --:--:-- 33826
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Ign:7 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 Packages
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 

In [ ]:
!edgetpu_compiler -s "/content/model.tflite"

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 2457 ms.

Input model: /content/model.tflite
Input size: 11.35MiB
Output model: model_edgetpu.tflite
Output size: 11.42MiB
On-chip memory used for caching model parameters: 19.75KiB
On-chip memory remaining for caching model parameters: 7.85MiB
Off-chip memory used for streaming uncached model parameters: 11.34MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 11
Operation log: model_edgetpu.log

Operator                       Count      Status

LOGISTIC                       1          Mapped to Edge TPU
RESHAPE                        1          Mapped to Edge TPU
SOFTMAX                        1          Mapped to Edge TPU
QUANTIZE                       3          Mapped to Edge TPU
CONV_2D                        1          Mapped to Edge TPU
FULLY_CONNECTED                4          Mapped to Edge TPU
Compilation child process completed within tim